In [2]:
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import os
import pandas as pd


 Grid dimensions 4608 x 3840 x 1


In [16]:

geom_file="/glade/work/afox/jedi/Jedi_experiments/input/wrf_hydro_nwm_files/conus/geometry_cfs_long_range_ens.nc"
ncfile = netCDF4.Dataset(geom_file, mode='r')

# Get metadata from the file
npx = ncfile.dimensions["west_east"].size
npy = ncfile.dimensions["south_north"].size
npz = 1

lons_conus = ncfile.variables["XLONG"][:]
lats_conus = ncfile.variables["XLAT"][:]

# Print field dimensions
print(" Grid dimensions", npx, 'x', npy, 'x', npz)

# Zero out array to fill with field
hgt = np.zeros((npy, npx))

hgt[:,:] = ncfile.variables["HGT"][:,:]

 Grid dimensions 4608 x 3840 x 1


In [10]:
make_obs_file = '/glade/work/afox/jedi/whj_test/build/wrf_hydro_nwm_jedi/test/Data/make_obs/make_obs_3d.swe_0000.nc'
ncfile = netCDF4.Dataset(make_obs_file, mode='r')
lons = ncfile['/MetaData/longitude'][:]
lats = ncfile['/MetaData/latitude'][:]
obs_err = ncfile['/ObsError/snow_water_equivalent'][:]
obs_val = ncfile['/ObsValue/snow_water_equivalent'][:]


In [3]:
obs_file = '/glade/u/home/afox/work/jedi/Jedi_experiments/ctest_letkf_oi_obs_swe.csv'
df = pd.read_csv(obs_file)
df

,StnObjID,StnID,ObsValue,ObsError,PreQC,dem_elev_m,rec_elev_m,latitude,longitude,datetime,variable_name
0,99999,XXXX1,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
1,99998,XXXX2,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
2,99997,XXXX3,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
3,99996,XXXX4,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
4,99995,XXXX5,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
5,99994,XXXX6,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
6,99993,XXXX7,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
7,99992,XXXX8,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
8,99991,XXXX9,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm
9,99990,XXXX10,9.999,5,0,9999.9999,9999.9999,99.9999,99.9999,2017-01-01T00:00:00Z,snow_water_equivalent_mm


In [11]:
df['ObsValue']=obs_val
df['ObsError']=obs_err
df['longitude']=lons
df['latitude']=lats

In [12]:
latitude = df['latitude']
longitude = df['longitude']

nobs = np.shape(longitude)
nobs = nobs[0]

In [17]:
lon_req = longitude
lat_req = latitude

array1 = np.asarray(lons_conus)
array2 = np.asarray(lats_conus)

height_out = np.zeros(nobs)

for i in range(nobs):
    lons_diff = (array1 - (lon_req[i]))**2
    lats_diff = (array2 - (lat_req[i]))**2
    comb_diff = lons_diff+lats_diff
    idx = comb_diff.argmin()
    height_out[i] = hgt.flatten()[idx]

In [18]:
df['dem_elev_m']=height_out
df['rec_elev_m']=height_out
df

,StnObjID,StnID,ObsValue,ObsError,PreQC,dem_elev_m,rec_elev_m,latitude,longitude,datetime,variable_name
0,99999,XXXX1,79.814255,10.0,0,3772.570312,3772.570312,39.043625,-106.477455,2017-01-01T00:00:00Z,snow_water_equivalent_mm
1,99998,XXXX2,152.080017,10.0,0,3401.420410,3401.420410,38.991238,-106.733734,2017-01-01T00:00:00Z,snow_water_equivalent_mm
2,99997,XXXX3,116.835526,10.0,0,3829.989258,3829.989258,39.028107,-106.581505,2017-01-01T00:00:00Z,snow_water_equivalent_mm
3,99996,XXXX4,101.906273,10.0,0,3671.916992,3671.916992,39.020054,-106.543938,2017-01-01T00:00:00Z,snow_water_equivalent_mm
4,99995,XXXX5,98.058578,10.0,0,3230.773926,3230.773926,38.963276,-106.592735,2017-01-01T00:00:00Z,snow_water_equivalent_mm
5,99994,XXXX6,105.810829,10.0,0,3087.497559,3087.497559,38.970039,-106.635300,2017-01-01T00:00:00Z,snow_water_equivalent_mm
6,99993,XXXX7,107.815788,10.0,0,3390.245117,3390.245117,38.975998,-106.556709,2017-01-01T00:00:00Z,snow_water_equivalent_mm
7,99992,XXXX8,94.997551,10.0,0,3087.497559,3087.497559,38.971184,-106.629829,2017-01-01T00:00:00Z,snow_water_equivalent_mm
8,99991,XXXX9,101.137108,10.0,0,3471.887695,3471.887695,38.975105,-106.600334,2017-01-01T00:00:00Z,snow_water_equivalent_mm
9,99990,XXXX10,121.853943,10.0,0,3104.223633,3104.223633,38.926815,-106.568909,2017-01-01T00:00:00Z,snow_water_equivalent_mm


In [19]:
df.to_csv (obs_file, index = False, header=True)